In [131]:
from PIL import Image, ImageFilter, ImageOps
from IPython.display import display
import numpy as np
import requests

def get_image_from_url(url):
    im = Image.open(requests.get(url, stream=True).raw)
    display(im)
    return im

def resize_based_on_width(im, basewidth = 100):
    wpercent = (basewidth / float(im.size[0]))
    hsize = int((float(im.size[1]) * float(wpercent)))
    im = im.resize((basewidth, hsize))
    return im

def preprocess(im):
    im = resize_based_on_width(im)
    im = im.quantize(5)
    im = im.convert("L")
    im = im.filter(ImageFilter.FIND_EDGES)
    im = ImageOps.invert(im)
    im = im.point( lambda p: 255 if p > 230 else 0 )
    im = im.convert('1')
    a = np.asarray(im)
    display(im)
    return a


In [134]:
def make_line(a, gap, target_color):
    dotted_line = []
    head = None
    for i in range(len(a) - 1):
        if head is None and a[i] == target_color:
            head = i
        if head is not None and a[i + 1] != target_color:
            tail = i
            dotted_line += [(head * gap, (tail + 1) * gap)]
            head = None
        if i == len(a) - 2 and a[i + 1] == target_color:
            if head is None:
                head = i + 1
            tail = i + 1
            dotted_line += [(head * gap, (tail + 1) * gap)]
    return dotted_line


def make_lines(a, gap=1, target_color=False):
    dotted_lines = []
    for a_ in a:
        dotted_lines += [make_line(a_, gap, target_color)]
    return dotted_lines


In [ ]:
def image_to_lines(path, url=None):
    if url is not None:
        im = get_image_from_url(url)
    else:
        im = Image.open(path)
    a = preprocess(im)
    lines = make_lines(a)
    return lines